<a href="https://colab.research.google.com/github/41371108H/114-1repo/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q google-generativeai

In [7]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [8]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('repo')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')


In [9]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1YmuPBSwZVq0m0-lwpsjCEcyXik3WVeE7a23sRin5ojw/edit?usp=sharing"
WORKSHEET_NAME = "工作表2"

REQUIRED_COLUMNS = ["姓名", "國文", "英文", "數學","總分", "總平均" ]

_auth_done = False
_gc = None
_ws = None

In [10]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import gradio as gr

# Google Sheets 認證
creds, _ = default()
gc = gspread.authorize(creds)

# 嘗試讀取 Google 試算表
try:
    gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YmuPBSwZVq0m0-lwpsjCEcyXik3WVeE7a23sRin5ojw/edit?usp=sharing')
    worksheet = gsheets.worksheet('工作表2')
    sheets = worksheet.get_all_values()
    df = pd.DataFrame(sheets[1:], columns=sheets[0])

    # 把成績欄位轉數字
    for col in ['國文', '英文', '數學', '總分', '平均分數']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

except gspread.exceptions.SpreadsheetNotFound:
    print("錯誤：找不到指定的 Google 試算表")
    df = pd.DataFrame(columns=['姓名', '國文', '英文', '數學', '總分', '平均分數'])
    worksheet = None


# 定義 Gradio 函式
def record_and_display(name, chinese, english, math):
    global df, worksheet

    if not name:
        return df, "⚠️ 請輸入姓名"

    # 計算總分和平均
    total = chinese + english + math
    average = round(total / 3, 2)

    # 建立紀錄
    record = {
        '姓名': name,
        '國文': chinese,
        '英文': english,
        '數學': math,
        '總分': total,
        '平均分數': average
    }

    # 加到本地 DataFrame
    new_row_df = pd.DataFrame([record])
    df = pd.concat([df, new_row_df], ignore_index=True)

    # 同步到 Google Sheets
    if worksheet:
        try:
            worksheet.append_row(list(record.values()))
            msg = f"✅ {name} 的成績已新增並同步至 Google 試算表。"
        except Exception as e:
            msg = f"⚠️ 無法寫入 Google 試算表：{e}"
    else:
        msg = f"✅ {name} 的成績已新增（僅儲存在本地）。"

    # 回傳 DataFrame 與訊息
    return df, msg


# 建立 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## 📝 學生成績紀錄系統")
    with gr.Row():
        name = gr.Textbox(label="學生姓名")
        chinese = gr.Number(label="國文成績", value=0)
        english = gr.Number(label="英文成績", value=0)
        math = gr.Number(label="數學成績", value=0)

    submit_btn = gr.Button("新增紀錄")
    output_df = gr.Dataframe(headers=['姓名', '國文', '英文', '數學', '總分', '平均分數'], interactive=False)
    msg = gr.Label()

    submit_btn.click(record_and_display, [name, chinese, english, math], [output_df, msg])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05893090246eb00d23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
